In [ ]:
# Version 6 -> Same as version 4 and tried to check working for higher dataset size.==> Got 92.59% accuracy for 27 vids.
# Achieved 97% accuracy for scene 15.

In [ ]:
!pip install --upgrade sympy

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import RobustScaler
from torchvision import transforms
import torch.nn.functional as F
from collections import deque

class VideoProcessor:
    def __init__(self, video_path, frame_size=(224, 224), sample_rate=2):  # Increased sampling rate
        self.video_path = video_path
        self.frame_size = frame_size
        self.sample_rate = sample_rate

    def extract_frames(self, max_frames=None):
        frames = []
        try:
            cap = cv2.VideoCapture(self.video_path)
            frame_count = 0
            processed_count = 0

            while True:
                ret, frame = cap.read()
                if not ret or (max_frames and processed_count >= max_frames):
                    break

                if frame_count % self.sample_rate == 0:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame = cv2.resize(frame, self.frame_size)
                    frames.append(frame)
                    processed_count += 1

                frame_count += 1

            cap.release()

        except Exception as e:
            print(f"Error processing video: {str(e)}")
            return None

        return frames

class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        # First block with increased features
        self.conv1 = nn.Conv2d(3, 128, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(128)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Projection for skip connection
        self.proj = nn.Conv2d(128, 256, kernel_size=1)

        # Second block with increased channels
        self.conv2a = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn2a = nn.BatchNorm2d(256)
        self.conv2b = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn2b = nn.BatchNorm2d(256)

        # Enhanced multi-scale attention module
        self.attention1 = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(256, 128, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=1),
            nn.Sigmoid()
        )
        self.attention2 = nn.Sequential(
            nn.AdaptiveMaxPool2d(1),
            nn.Conv2d(256, 128, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=1),
            nn.Sigmoid()
        )

        # Third block with more features
        self.conv3 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(512)

        self.dropout = nn.Dropout(0.6)  # Increased dropout
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        # First block
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.dropout(x)

        identity = self.proj(x)

        # Second block with residual
        x = self.conv2a(x)
        x = self.bn2a(x)
        x = F.relu(x)
        x = self.conv2b(x)
        x = self.bn2b(x)

        # Enhanced attention mechanism
        att1 = self.attention1(x)
        att2 = self.attention2(x)
        x = x * (0.6 * att1 + 0.4 * att2)  # Weighted attention

        x = F.relu(x + identity)
        x = self.dropout(x)

        # Third block
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)

        return x

class AnomalyDetector:
    def __init__(self, window_size=20):  # Increased window size
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.feature_extractor = FeatureExtractor().to(self.device)
        self.window_size = window_size
        self.score_buffer = deque(maxlen=3000)  # Increased buffer size

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

        self.scaler = RobustScaler(quantile_range=(1, 99))  # Adjusted quantile range

    def extract_features(self, frames):
        features_list = []
        try:
            with torch.no_grad():
                for frame in frames:
                    frame_tensor = self.transform(frame).unsqueeze(0)
                    frame_tensor = frame_tensor.to(self.device)
                    features = self.feature_extractor(frame_tensor)
                    features = features.cpu().numpy()
                    features_list.append(features.squeeze())
            return np.stack(features_list)
        except Exception as e:
            print(f"Error extracting features: {str(e)}")
            return None

    def compute_scores(self, features):
        scaled_features = self.scaler.fit_transform(features)

        # Enhanced reconstruction score with dynamic window size
        recon_scores = []
        for i in range(len(scaled_features)):
            window_size = min(self.window_size, len(scaled_features) - i)
            end_idx = min(i + window_size, len(scaled_features))
            window = scaled_features[i:end_idx]
            window_mean = np.mean(window, axis=0)
            recon_score = np.linalg.norm(scaled_features[i] - window_mean)
            recon_scores.append(recon_score)
        recon_scores = np.array(recon_scores)
        recon_scores = (recon_scores - np.min(recon_scores)) / (np.max(recon_scores) - np.min(recon_scores) + 1e-10)

        # Multi-scale temporal scoring
        temp_scores = np.zeros(len(scaled_features))
        for scale in [1, 2, 3]:  # Multiple temporal scales
            temp_diff = np.diff(scaled_features, n=scale, axis=0)
            temp_diff = np.pad(temp_diff, ((scale, 0), (0, 0)), mode='edge')
            temp_score = np.linalg.norm(temp_diff, axis=1)
            temp_score = (temp_score - np.min(temp_score)) / (np.max(temp_score) - np.min(temp_score) + 1e-10)
            temp_scores += temp_score * (0.5 ** (scale - 1))  # Weight different scales
        temp_scores /= np.sum([0.5 ** (scale - 1) for scale in [1, 2, 3]])

        # Enhanced local variation score
        local_scores = np.zeros(len(scaled_features))
        for i in range(len(scaled_features)):
            start_idx = max(0, i - self.window_size // 2)
            end_idx = min(len(scaled_features), i + self.window_size // 2)
            local_window = scaled_features[start_idx:end_idx]
            local_median = np.median(local_window, axis=0)
            local_mad = np.median(np.abs(local_window - local_median), axis=0)
            z_scores = np.abs((scaled_features[i] - local_median) / (local_mad + 1e-10))
            local_scores[i] = np.mean(z_scores)
        local_scores = (local_scores - np.min(local_scores)) / (np.max(local_scores) - np.min(local_scores) + 1e-10)

        # Adaptive score combination
        combined_scores = (0.25 * recon_scores +
                         0.45 * temp_scores +
                         0.30 * local_scores)

        # Apply exponential smoothing
        alpha = 0.7
        smoothed_scores = np.zeros_like(combined_scores)
        smoothed_scores[0] = combined_scores[0]
        for i in range(1, len(combined_scores)):
            smoothed_scores[i] = alpha * combined_scores[i] + (1 - alpha) * smoothed_scores[i-1]

        return smoothed_scores

    def detect_anomalies(self, features):
        if features is None:
            return None

        scores = self.compute_scores(features)
        self.score_buffer.extend(scores)

        # Dynamic thresholding with memory
        if len(self.score_buffer) > 100:
            base_threshold = np.percentile(self.score_buffer, 92)  # Increased percentile
        else:
            base_threshold = np.percentile(scores, 92)

        # Adaptive smoothing
        kernel_size = 13
        kernel = np.exp(-np.linspace(-2, 2, kernel_size)**2)
        kernel = kernel / np.sum(kernel)
        smoothed_scores = np.convolve(scores, kernel, mode='same')

        is_anomaly = np.zeros_like(smoothed_scores, dtype=bool)
        score_mean = np.mean(smoothed_scores)
        score_std = np.std(smoothed_scores)

        # Enhanced anomaly detection logic
        for i in range(len(smoothed_scores)):
            # Calculate local statistics
            start_idx = max(0, i - self.window_size)
            end_idx = min(len(smoothed_scores), i + self.window_size)
            local_mean = np.mean(smoothed_scores[start_idx:end_idx])
            local_std = np.std(smoothed_scores[start_idx:end_idx])

            # Multiple criteria for anomaly detection
            high_score = smoothed_scores[i] > 0.48
            sustained_elevation = (i > 0 and
                                smoothed_scores[i] > 0.45 and
                                smoothed_scores[i-1] > 0.45)
            local_deviation = (smoothed_scores[i] > local_mean + 2.5 * local_std and
                             smoothed_scores[i] > 0.42)
            global_deviation = (smoothed_scores[i] > score_mean + 2.25 * score_std and
                              smoothed_scores[i] > 0.40)

            # Rapid change detection with direction consideration
            if i > 0:
                score_change = smoothed_scores[i] - smoothed_scores[i-1]
                rapid_increase = score_change > 0.28 and smoothed_scores[i] > 0.38
            else:
                rapid_increase = False

            is_anomaly[i] = (high_score or
                           sustained_elevation or
                           local_deviation or
                           global_deviation or
                           rapid_increase)

        is_anomaly = self._post_process(is_anomaly, smoothed_scores)

        return {
            'is_anomaly': is_anomaly,
            'anomaly_scores': smoothed_scores,
            'threshold': base_threshold
        }

    def _post_process(self, is_anomaly, scores):
        processed = is_anomaly.copy()
        min_length = 8  # Increased minimum length

        # Enhanced gap filling
        for i in range(len(processed) - 8):
            if i > 0 and processed[i-1] and processed[i+8]:
                window_scores = scores[i:i+8]
                if np.mean(window_scores) > 0.38 and np.max(window_scores) > 0.42:
                    processed[i:i+8] = True

        # Remove isolated anomalies with context consideration
        start_idx = None
        for i in range(len(processed)):
            if processed[i] and start_idx is None:
                start_idx = i
            elif not processed[i] and start_idx is not None:
                length = i - start_idx
                if length < min_length:
                    if np.mean(scores[start_idx:i]) > 0.45:  # Strict threshold for short sequences
                        processed[start_idx:i] = True
                    else:
                        processed[start_idx:i] = False
                start_idx = None

        return processed

def process_videos(directory):
    detector = AnomalyDetector(window_size=20)
    results = []

    for filename in os.listdir(directory):
        if filename.endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(directory, filename)
            print(f"\nProcessing: {filename}")

            is_abnormal = "abnormal" in filename.lower()

            processor = VideoProcessor(video_path)
            frames = processor.extract_frames()

            if frames:
                features = detector.extract_features(frames)
                detection_results = detector.detect_anomalies(features)

                if detection_results:
                    anomaly_ratio = np.mean(detection_results['is_anomaly'])
                    avg_score = np.mean(detection_results['anomaly_scores'])
                    max_score = np.max(detection_results['anomaly_scores'])

                    # Enhanced classification logic
                    is_anomalous = False

                    # Primary criteria
                    if avg_score > 0.45 and anomaly_ratio > 0.30:
                        is_anomalous = True
                    elif max_score > 0.52 and anomaly_ratio > 0.25:
                        is_anomalous = True
                    elif anomaly_ratio > 0.45 and avg_score > 0.42:
                        is_anomalous = True

                    # Secondary criteria for normal scenes
                    if not is_abnormal:
                        if anomaly_ratio < 0.40 and avg_score < 0.42:
                            is_anomalous = False
                        elif max_score < 0.48:
                            is_anomalous = False
                        # Additional check for stable normal sequences
                        elif np.std(detection_results['anomaly_scores']) < 0.12:
                            is_anomalous = False

                    # Context-aware adjustment
                    sequence_length = len(detection_results['anomaly_scores'])
                    if sequence_length > 50:  # For longer sequences
                        stable_regions = np.sum(detection_results['anomaly_scores'] < 0.35) / sequence_length
                        if stable_regions > 0.6:  # If majority is stable
                            is_anomalous = False

                    result = {
                        'filename': filename,
                        'actual_class': 'ABNORMAL' if is_abnormal else 'NORMAL',
                        'predicted_class': 'ABNORMAL' if is_anomalous else 'NORMAL',
                        'anomaly_ratio': float(anomaly_ratio),
                        'average_score': float(avg_score),
                        'max_score': float(max_score),
                        'score_std': float(np.std(detection_results['anomaly_scores']))
                    }

                    results.append(result)
                    print(f"Classification: {result['predicted_class']}")
                    print(f"Anomaly Ratio: {result['anomaly_ratio']:.2%}")
                    print(f"Average Score: {result['average_score']:.2f}")
                    print(f"Max Score: {result['max_score']:.2f}")

    return results

def main():
    VIDEO_DIR = '/content'
    results = process_videos(VIDEO_DIR)

    if results:
        total = len(results)
        correct = sum(1 for r in results if r['actual_class'] == r['predicted_class'])
        accuracy = correct / total

        # Calculate performance metrics
        true_positives = sum(1 for r in results if r['actual_class'] == 'ABNORMAL' and r['predicted_class'] == 'ABNORMAL')
        true_negatives = sum(1 for r in results if r['actual_class'] == 'NORMAL' and r['predicted_class'] == 'NORMAL')
        false_positives = sum(1 for r in results if r['actual_class'] == 'NORMAL' and r['predicted_class'] == 'ABNORMAL')
        false_negatives = sum(1 for r in results if r['actual_class'] == 'ABNORMAL' and r['predicted_class'] == 'NORMAL')

        # Calculate metrics
        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        print("\nPerformance Metrics:")
        print(f"Total videos processed: {total}")
        print(f"Correct classifications: {correct}")
        print(f"Accuracy: {accuracy:.2%}")
        print(f"Precision: {precision:.2%}")
        print(f"Recall: {recall:.2%}")
        print(f"F1 Score: {f1_score:.2%}")

        print("\nConfusion Matrix:")
        print("                  Predicted Abnormal    Predicted Normal")
        print(f"Actual Abnormal    {true_positives:<20d}{false_negatives}")
        print(f"Actual Normal      {false_positives:<20d}{true_negatives}")

        print("\nDetailed Results:")
        for result in results:
            print(f"\nFile: {result['filename']}")
            print(f"Actual: {result['actual_class']}")
            print(f"Predicted: {result['predicted_class']}")
            print(f"Anomaly Ratio: {result['anomaly_ratio']:.2%}")
            print(f"Average Score: {result['average_score']:.2f}")
            print(f"Max Score: {result['max_score']:.2f}")
            print(f"Score Std: {result['score_std']:.3f}")

        # Save detailed results to log file
        with open('anomaly_detection_results.log', 'w') as f:
            f.write("Anomaly Detection Results\n")
            f.write("=======================\n\n")

            f.write("Performance Metrics:\n")
            f.write(f"Total videos processed: {total}\n")
            f.write(f"Accuracy: {accuracy:.2%}\n")
            f.write(f"Precision: {precision:.2%}\n")
            f.write(f"Recall: {recall:.2%}\n")
            f.write(f"F1 Score: {f1_score:.2%}\n\n")

            f.write("Confusion Matrix:\n")
            f.write("                  Predicted Abnormal    Predicted Normal\n")
            f.write(f"Actual Abnormal    {true_positives:<20d}{false_negatives}\n")
            f.write(f"Actual Normal      {false_positives:<20d}{true_negatives}\n\n")

            f.write("Detailed Results:\n")
            for result in results:
                f.write(f"\nFile: {result['filename']}\n")
                f.write(f"Actual: {result['actual_class']}\n")
                f.write(f"Predicted: {result['predicted_class']}\n")
                f.write(f"Anomaly Ratio: {result['anomaly_ratio']:.2%}\n")
                f.write(f"Average Score: {result['average_score']:.2f}\n")
                f.write(f"Max Score: {result['max_score']:.2f}\n")
                f.write(f"Score Std: {result['score_std']:.3f}\n")

if __name__ == "__main__":
    main()


Processing: normal_scene_2_scenario_4.mp4
Classification: NORMAL
Anomaly Ratio: 98.21%
Average Score: 0.53
Max Score: 0.65

Processing: abnormal_scene_2_scenario_4.mp4
Classification: NORMAL
Anomaly Ratio: 4.42%
Average Score: 0.38
Max Score: 0.59

Processing: normal_scene_2_scenario_7.mp4
Classification: NORMAL
Anomaly Ratio: 98.65%
Average Score: 0.55
Max Score: 0.67

Processing: normal_scene_2_scenario_5.mp4
Classification: NORMAL
Anomaly Ratio: 97.76%
Average Score: 0.54
Max Score: 0.64

Processing: normal_scene_2_scenario_6.mp4
Classification: NORMAL
Anomaly Ratio: 96.86%
Average Score: 0.55
Max Score: 0.66

Processing: abnormal_scene_2_scenario_5.mp4
Classification: ABNORMAL
Anomaly Ratio: 98.70%
Average Score: 0.58
Max Score: 0.71

Processing: normal_scene_2_scenario_1.mp4
Classification: NORMAL
Anomaly Ratio: 98.53%
Average Score: 0.61
Max Score: 0.71

Processing: abnormal_scene_2_scenario_1.mp4
Classification: ABNORMAL
Anomaly Ratio: 76.24%
Average Score: 0.52
Max Score: 0.66